# ------------------------------------生成对抗网络-------------------------------------------------

- 包含生成器和判别器，相辅相成一同进化

In [2]:
import os

import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

# ------------------------------------生成对抗网络-------------------------------------------------

# ------------------------------------准备数据集-------------------------------------------------

In [5]:
"""创建样本文件夹"""
# 检查有无文件夹，没有则创建
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [6]:
"""定义图片处理操作"""
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [8]:
"""加载MNSIT数据集"""
mnist = torchvision.datasets.MNIST(root='data/', train=True, transform=transform, download=True)

100%|███████████████████████████████████████████████████████████████████| 9912422/9912422 [00:07<00:00, 1283875.47it/s]


Extracting data/MNIST\raw\train-images-idx3-ubyte.gz to data/MNIST\raw



100%|████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 137679.39it/s]


Extracting data/MNIST\raw\train-labels-idx1-ubyte.gz to data/MNIST\raw



100%|████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:02<00:00, 774822.51it/s]


Extracting data/MNIST\raw\t10k-images-idx3-ubyte.gz to data/MNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<?, ?it/s]


Extracting data/MNIST\raw\t10k-labels-idx1-ubyte.gz to data/MNIST\raw



In [9]:
"""打包数据用于训练"""
data_loader = torch.utils.data.DataLoader(dataset=mnist, batch_size=batch_size, shuffle=True)

# ------------------------------------定义网络模型-------------------------------------------------
- 模型分为生成器和判别器两个模型

In [3]:
"""定义使用GPU计算"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
"""定义判别器模型"""
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()
)
# 将模型转移到GPU
D = D.to(device)

In [16]:
"""定义生成器模型"""
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh()
)
# 将模型转移到GPU
G = G.to(device)

In [17]:
"""定义损失函数和参数优化器"""
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [18]:
"""将张量的值规范化"""
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

# ------------------------------------定义超参数-------------------------------------------------

In [4]:
"""设置超参数"""
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'sample'

# ------------------------------------开始训练-------------------------------------------------

In [23]:
"""训练模型"""
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        # 将图片进行调整，并转移到gpu中
        images = images.reshape(batch_size, -1).to(device)
        
        # 生成真实和虚假的标签
        # 标签只有真假之分不需要对应的数字了，所以直接生成就可以！
        real_labels = torch.ones(batch_size, 1).to(device) # 真的图片标签都是1
        fake_labels = torch.zeros(batch_size, 1).to(device) # 假的图片标签都是0
        
        
        # 训练判别器
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # 训练生成器
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        g_loss = criterion(outputs, real_labels)
        
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
        
    # 保存真实图片
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # 保存生成的样本图片
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))
        

Epoch [0/200], Step [200/600], d_loss: 0.2794, g_loss: 3.4786, D(x): 0.92, D(G(z)): 0.15
Epoch [0/200], Step [400/600], d_loss: 0.3433, g_loss: 3.3180, D(x): 0.88, D(G(z)): 0.11
Epoch [0/200], Step [600/600], d_loss: 0.4557, g_loss: 3.9202, D(x): 0.77, D(G(z)): 0.08
Epoch [1/200], Step [200/600], d_loss: 0.2103, g_loss: 3.4889, D(x): 0.93, D(G(z)): 0.10
Epoch [1/200], Step [400/600], d_loss: 0.4969, g_loss: 2.9491, D(x): 0.83, D(G(z)): 0.17
Epoch [1/200], Step [600/600], d_loss: 0.4181, g_loss: 3.1537, D(x): 0.81, D(G(z)): 0.06
Epoch [2/200], Step [200/600], d_loss: 0.3252, g_loss: 2.8327, D(x): 0.92, D(G(z)): 0.16
Epoch [2/200], Step [400/600], d_loss: 0.8886, g_loss: 2.3544, D(x): 0.71, D(G(z)): 0.27
Epoch [2/200], Step [600/600], d_loss: 0.5131, g_loss: 2.5357, D(x): 0.82, D(G(z)): 0.19
Epoch [3/200], Step [200/600], d_loss: 0.5854, g_loss: 2.4500, D(x): 0.80, D(G(z)): 0.17
Epoch [3/200], Step [400/600], d_loss: 0.4032, g_loss: 2.8813, D(x): 0.88, D(G(z)): 0.14
Epoch [3/200], Step [

Epoch [30/200], Step [600/600], d_loss: 0.4635, g_loss: 4.6277, D(x): 0.88, D(G(z)): 0.14
Epoch [31/200], Step [200/600], d_loss: 0.5610, g_loss: 3.1184, D(x): 0.96, D(G(z)): 0.32
Epoch [31/200], Step [400/600], d_loss: 0.3943, g_loss: 3.2610, D(x): 0.86, D(G(z)): 0.08
Epoch [31/200], Step [600/600], d_loss: 0.3821, g_loss: 2.6224, D(x): 0.89, D(G(z)): 0.13
Epoch [32/200], Step [200/600], d_loss: 0.3715, g_loss: 3.7759, D(x): 0.87, D(G(z)): 0.11
Epoch [32/200], Step [400/600], d_loss: 0.4365, g_loss: 4.0775, D(x): 0.86, D(G(z)): 0.12
Epoch [32/200], Step [600/600], d_loss: 0.2625, g_loss: 4.2929, D(x): 0.90, D(G(z)): 0.09
Epoch [33/200], Step [200/600], d_loss: 0.3368, g_loss: 4.7470, D(x): 0.86, D(G(z)): 0.05
Epoch [33/200], Step [400/600], d_loss: 0.3950, g_loss: 3.8537, D(x): 0.89, D(G(z)): 0.15
Epoch [33/200], Step [600/600], d_loss: 0.5643, g_loss: 4.2796, D(x): 0.83, D(G(z)): 0.07
Epoch [34/200], Step [200/600], d_loss: 0.3852, g_loss: 4.7360, D(x): 0.89, D(G(z)): 0.08
Epoch [34/

Epoch [61/200], Step [400/600], d_loss: 0.6353, g_loss: 2.0085, D(x): 0.74, D(G(z)): 0.15
Epoch [61/200], Step [600/600], d_loss: 0.5898, g_loss: 2.0913, D(x): 0.76, D(G(z)): 0.16
Epoch [62/200], Step [200/600], d_loss: 0.6879, g_loss: 2.6199, D(x): 0.71, D(G(z)): 0.14
Epoch [62/200], Step [400/600], d_loss: 0.7294, g_loss: 1.9592, D(x): 0.77, D(G(z)): 0.24
Epoch [62/200], Step [600/600], d_loss: 0.5393, g_loss: 2.1617, D(x): 0.81, D(G(z)): 0.16
Epoch [63/200], Step [200/600], d_loss: 0.6856, g_loss: 2.5648, D(x): 0.75, D(G(z)): 0.19
Epoch [63/200], Step [400/600], d_loss: 0.8805, g_loss: 1.9476, D(x): 0.80, D(G(z)): 0.33
Epoch [63/200], Step [600/600], d_loss: 0.8309, g_loss: 1.8976, D(x): 0.76, D(G(z)): 0.29
Epoch [64/200], Step [200/600], d_loss: 0.8854, g_loss: 2.1233, D(x): 0.76, D(G(z)): 0.30
Epoch [64/200], Step [400/600], d_loss: 0.7388, g_loss: 1.9180, D(x): 0.73, D(G(z)): 0.19
Epoch [64/200], Step [600/600], d_loss: 0.7430, g_loss: 1.5674, D(x): 0.74, D(G(z)): 0.19
Epoch [65/

KeyboardInterrupt: 

In [25]:
# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')